# Семинар 4
## Сбор характеристик каналов

In [55]:
import pathlib
import yaml

ROOT = pathlib.Path().resolve().parent.parent

with open(ROOT / "config.yml", "r") as f:
    config = yaml.safe_load(f)

In [56]:
# 3
DATA_PATH = ROOT / config['api']['DATA_PATH'] / 'seminar4'

query = "корейская косметика"
type_query = "channel"

# Для поиска видео с учетом даты:
publishedAfter = '2020-01-01T00:00:00Z'
publishedBefore = '2022-01-01T00:00:00Z'

In [57]:
# 3.1
class KeyStorage:
    """
    Class to use in iteration
    """

    def __init__(self, key_arr: list[str]):
        self.key_gen = iter(key_arr)

    def get_next_key(self):
        """
        Returns API key to use
        :return:
        """
        return next(self.key_gen)


key_storage = KeyStorage(config['api']['keys'])

In [58]:
# 3.2 Просмотр API ключа
API_KEY = key_storage.get_next_key()

In [59]:
#4
import googleapiclient.discovery as api

In [60]:
#5
api_service_name = "youtube"
api_version = "v3"

youtube = api.build(api_service_name,
                    api_version,
                    developerKey=API_KEY)

In [61]:
# Открываем свой датафрейм

In [62]:
# 7
import pandas as pd

data = pd.read_excel(DATA_PATH / 'Channel_Not_sorted+Sorted+Date.xlsx')
data.head()

,Unnamed: 0,etag,id.kind,id.channelId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.medium.url,...,snippet.liveBroadcastContent,snippet.publishTime,id.videoId,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.width,snippet.thumbnails.high.height,kind
0,youtube#searchResult,3DyCGRAji4vs67_K6Xodsfy_yRc,youtube#channel,UClTe1W6iuivE5R1XBacrENg,2022,UClTe1W6iuivE5R1XBacrENg,Корейская косметика,Корейская косметика на прямую от производителя...,https://yt3.ggpht.com/tnzcStGWX_50UZA6gPqMmGJJ...,https://yt3.ggpht.com/tnzcStGWX_50UZA6gPqMmGJJ...,...,none,2022-10-20T13:03:39Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,youtube#searchResult,KD4wB0kFCleorLAZUqunFeeNnGU,youtube#channel,UCyBkYEimyewSS9Tf_XasxqA,2019,UCyBkYEimyewSS9Tf_XasxqA,Chok - Chok Корейская косметика,1.,https://yt3.ggpht.com/ytc/AL5GRJXSaaDeBjL92F2Y...,https://yt3.ggpht.com/ytc/AL5GRJXSaaDeBjL92F2Y...,...,none,2019-09-26T16:51:01Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,youtube#searchResult,rYW08nw-AIqP6rHhabVqo3r4GOo,youtube#channel,UCPmSyu4_Xxw2d8uOThJ5tCQ,2015,UCPmSyu4_Xxw2d8uOThJ5tCQ,Корейская косметика,NaN,https://yt3.ggpht.com/ytc/AL5GRJX9KQ-jcjyvtAC8...,https://yt3.ggpht.com/ytc/AL5GRJX9KQ-jcjyvtAC8...,...,none,2015-04-01T11:37:55Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,youtube#searchResult,hq-V8AVxdQs4HTDfyp9T7YaCWak,youtube#channel,UCIH9WX73cmksrcE4lKzns2w,2021,UCIH9WX73cmksrcE4lKzns2w,Корейская люкс косметика,Корейская люкс косметика Здоровая кожа - залог...,https://yt3.ggpht.com/HE9yWbxQN3MDaXFmyJeytKHc...,https://yt3.ggpht.com/HE9yWbxQN3MDaXFmyJeytKHc...,...,none,2021-01-11T04:37:21Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,youtube#searchResult,26Gb5w9PCwul3ZBjH3XPbWR_beY,youtube#channel,UCFmgLIGI4CTB_g7LEw_VXfg,2018,UCFmgLIGI4CTB_g7LEw_VXfg,Корейская Косметика,NaN,https://yt3.ggpht.com/ytc/AL5GRJX5-fC0j0POMv5F...,https://yt3.ggpht.com/ytc/AL5GRJX5-fC0j0POMv5F...,...,none,2018-05-16T08:08:08Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
# 8

search_parts = [
    "brandingSettings",
    "contentDetails",
    "contentOwnerDetails",
    "localizations",
    "snippet",
    "statistics",
    "topicDetails",
]

channel_ids = list(data['id.channelId'])

In [85]:
# 9 Одиночный вызов API для одного канала
request = youtube.channels().list(
    part=search_parts,
    maxResults=50,
    id=channel_ids[:50],
)
response = request.execute()

df_supplemented = pd.json_normalize(response['items'])
df_supplemented.head()

,kind,etag,id,snippet.title,snippet.description,snippet.customUrl,snippet.publishedAt,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,brandingSettings.channel.trackingAnalyticsAccountId,brandingSettings.channel.unsubscribedTrailer,brandingSettings.channel.country,brandingSettings.image.bannerExternalUrl,snippet.defaultLanguage,brandingSettings.channel.defaultLanguage,localizations.en_US.title,localizations.en_US.description,localizations.ru_RU.title,localizations.ru_RU.description
0,youtube#channel,M7HU1NBc0yOLV9d82rWNL2HjMes,UChzXOCsMyxz-JGpt_wfS0lA,FERIKA корейская косметика,www.ferika.ru - это надежный и удобный интерне...,@ferika5494,2020-02-27T09:56:08.649894Z,https://yt3.ggpht.com/ytc/AL5GRJX98EYaRoFqrP8Q...,88,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,youtube#channel,nvDVzAcauMEf_ZMjCfm_n6g5KXY,UClfGcpJdnc7ClcovZoalhBw,LuckyCosmetics Корейская косметика,"Всем привет!\nСпасибо, что зашли на канал инте...",@luckycosmetics8244,2017-12-29T16:25:58Z,https://yt3.ggpht.com/ytc/AL5GRJXjRWvsJE8CJBOu...,88,88,...,UA-15256516-4,-2Buo_2kXJg,RU,https://lh3.googleusercontent.com/6hx_JISaHdoa...,NaN,NaN,NaN,NaN,NaN,NaN
2,youtube#channel,Z7aWOD_b18D5-hwxQoMqx4HfQP0,UC_faj8TvjRJj9IbQHkUyXNw,Корейская Косметика,,@user-bf5in5qi7e,2019-03-02T13:34:25Z,https://yt3.ggpht.com/ytc/AL5GRJUiv0F7HNErt8Ml...,88,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,youtube#channel,_Um5kwe_xtezX930hwlQDyLfaKY,UCIH9WX73cmksrcE4lKzns2w,Корейская люкс косметика,• ────── 🇰🇷 ────── •\nКорейская люкс косметика...,@user-ym2ll1io4l,2021-01-11T04:37:21.874233Z,https://yt3.ggpht.com/HE9yWbxQN3MDaXFmyJeytKHc...,88,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,youtube#channel,Ea2ayVIENjzeynyJpijJsdZpKjM,UCheZqVGVn9dNroNDLtvxTww,Atomy. Атоми Корейская косметика. Здоровье,,@atomy..3946,2020-09-27T10:10:01.503234Z,https://yt3.ggpht.com/ytc/AL5GRJUlzJHyU9EBXaRL...,88,88,...,NaN,NaN,RU,https://yt3.googleusercontent.com/IZo3G6McI3Ez...,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
# 10 Множественный вызов по 50 элементов

df_supplemented = pd.DataFrame()

for i in range(0, len(data), 50):
    # границы интервала
    s, e = i, i+50
    if e > len(data): e = len(data)

    request = youtube.channels().list(
    part=search_parts,
    maxResults=50,
    id=channel_ids[s:e],
    )
    response = request.execute()

    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])
    print(f"Итерация {i // 50} завершена")

Итерация 0 завершена
Итерация 1 завершена
Итерация 2 завершена
Итерация 3 завершена
Итерация 4 завершена
Итерация 5 завершена
Итерация 6 завершена
Итерация 7 завершена
Итерация 8 завершена
Итерация 9 завершена
Итерация 10 завершена
Итерация 11 завершена
Итерация 12 завершена
Итерация 13 завершена
Итерация 14 завершена
Итерация 15 завершена
Итерация 16 завершена
Итерация 17 завершена
Итерация 18 завершена
Итерация 19 завершена
Итерация 20 завершена
Итерация 21 завершена
Итерация 22 завершена
Итерация 23 завершена
Итерация 24 завершена
Итерация 25 завершена
Итерация 26 завершена
Итерация 27 завершена
Итерация 28 завершена
Итерация 29 завершена
Итерация 30 завершена
Итерация 31 завершена
Итерация 32 завершена
Итерация 33 завершена
Итерация 34 завершена
Итерация 35 завершена


In [91]:
# 11 Просмотр параметров датафрейма
df_supplemented.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1757 entries, 0 to 16
Data columns (total 50 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   kind                                                 1757 non-null   object
 1   etag                                                 1757 non-null   object
 2   id                                                   1757 non-null   object
 3   snippet.title                                        1757 non-null   object
 4   snippet.description                                  1757 non-null   object
 5   snippet.customUrl                                    1757 non-null   object
 6   snippet.publishedAt                                  1757 non-null   object
 7   snippet.thumbnails.default.url                       1757 non-null   object
 8   snippet.thumbnails.default.width                     1757 non-null   int64 
 9  

In [93]:
# 12 Валидация количества каналов

if len(data) == len(df_supplemented): print('Всё ок')
else: print("Не досчитались:", abs(len(data) - len(df_supplemented)), "каналов")

Не досчитались: 10 каналов


In [107]:
# 13 Просмотр статистики по каналам

list_of_col = [x for x in df_supplemented.columns if x.startswith('stat')]

for col in list_of_col:
    df_supplemented[col] = pd.to_numeric(df_supplemented[col],downcast="integer")

In [113]:
for col in list_of_col:
    print(f"Параметр {col}")
    print(f"Average - {df_supplemented[col].mean()}\n"
          f"Minimal - {df_supplemented[col].min()},\n"
          f"Maximum - {df_supplemented[col].max()}\n"
          f"Median - {df_supplemented[col].median()}")
    print("="*15)

Параметр statistics.viewCount
Average - 3108456.5702902675
Minimal - 0,
Maximum - 447355047
Median - 2840.0
Параметр statistics.subscriberCount
Average - 21112.29766647695
Minimal - 0,
Maximum - 3030000
Median - 29.0
Параметр statistics.hiddenSubscriberCount
Average - 0.0
Minimal - False,
Maximum - False
Median - 0.0
Параметр statistics.videoCount
Average - 48.64484917472965
Minimal - 0,
Maximum - 953
Median - 13.0


In [114]:
# 13 сохранение файлов в специальной директории
import os

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

df_supplemented.to_excel(DATA_PATH / f'{type_query.capitalize()}_characteristics.xlsx', index=False)